In [ ]:
# ==================================================================================================================
# ===========================================> Instalación librerías <==============================================
# ==================================================================================================================

#%pip install pandas    # NO HACE FALTA INSTALAR
#%pip install numpy     # NO HACE FALTA INSTALAR
#%pip install pickle    # NO HACE FALTA INSTALAR
#%pip install sklearn   # NO HACE FALTA INSTALAR

#%pip install scikit-learn          # DESCOMENTAR LA PRIMERA VEZ
#%pip install category_encoders     # DESCOMENTAR LA PRIMERA VEZ
#%pip install lightgbm              # DESCOMENTAR LA PRIMERA VEZ
#%pip install matplotlib            # DESCOMENTAR LA PRIMERA VEZ
#%pip install seaborn               # DESCOMENTAR LA PRIMERA VEZ



In [ ]:
# ==================================================================================================================
# ===========================================> Carga librerías <====================================================
# ==================================================================================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# ==================================================================================================================
# ===========================================> Carga Datos <========================================================
# ==================================================================================================================

df = pd.read_csv('../data/train.csv', low_memory=False)
#df = df.sample(10000)

# =====================> EDA

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración número de columnas <=====================================
# ==================================================================================================================

len(df.columns)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col numericas <=====================
# ==================================================================================================================

df.describe()

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col objectos <=====================
# ==================================================================================================================

df.describe(include=object)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración 20 primeras filas <======================================
# ==================================================================================================================

df.head(20)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración tipos <==================================================
# ==================================================================================================================

df.info()

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración nulos <==================================================
# ==================================================================================================================

df.isna().sum().sort_values(ascending=False)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración valores y estadísticas <=================================
# ==================================================================================================================

stats = []

for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    
stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique Values', 'Missing Values %', 'Biggest Category %', 'Type'])
stats_df.sort_values('Missing Values %', ascending=False).head(20)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar valores nulos <=============================================
# ==================================================================================================================

drop_cols_min = 0.6
drop_cols = []

for c in df.columns:
    if df[c].isna().sum() / len(df) > drop_cols_min:
        drop_cols.append(c)
        
for c in drop_cols:
    df = df.drop(c, axis=1)

print(drop_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 3 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 2).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 4 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 3).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 5 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 4).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4", c + "_5"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 6 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 5).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4", c + "_5", c + "_6"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(mask_cols)